# **Importing data & pre-processing**
Import data from GitHub & set row names (same as in part1):

In [2]:
# import file with NOT normalized expression data
dat.abundances <- read.table("https://raw.githubusercontent.com/ddz-icb/OmicsDataAnalysisCourse/main/data/dat.abundances.txt",
                            header=T,
                            sep="\t")
rownames(dat.abundances) <- dat.abundances[,1]        # set rownames to IDs from first column
dat.abundances <- data.matrix(dat.abundances[,-1])    # delete first column and change "data frame" to numeric "data matrix"



# import file with normalized data and extended information
dat.ext <- read.table("https://raw.githubusercontent.com/ddz-icb/OmicsDataAnalysisCourse/main/data/dat.ext.txt",
                            header=T,
                            sep="\t")

Keep only phosphorylated peptides (same as in part1):

In [3]:
# give row numbers with phosphopeptides
phospep.idx <- grep("Phospho", dat.ext$Modifications)   # grep() gives all row numbers containing the given pattern in column 'Modifications'

# keep only phospopeptides
dat.abundances <- dat.abundances[phospep.idx,]
#dat.nonorm <- dat.nonorm[phospep.idx,]
dat.ext <- dat.ext[phospep.idx,]

In part1, we have determined that the normalization results of the device software are OK and that we can use them. Therefore, we do not need to perform our own raw data normalization. However, we want to perform a group-specific imputation and replace isolated missing values to avoid excluding almost completely quantified phosphopeptides in some analysis steps. For this, the same imputation as in part1 is performed.

In [ ]:
# Give row vectors with group-specific column numbers
basal.idx <- grep("Basal", colnames(dat.abundances))
insulin.idx <- grep("Insulin", colnames(dat.abundances))



dat.abundances2 <- dat.abundances
for(i in 1:nrow(dat.abundances2)){
    if(sum(is.na(dat.abundances2[i,basal.idx])) == 1){
      na.idx <- which(is.na(dat.abundances2[i,basal.idx]))
      dat.abundances2[i, basal.idx[na.idx]] <- mean(dat.abundances2[i, basal.idx], na.rm=T)
    }

    if(sum(is.na(dat.abundances2[i,insulin.idx])) == 1){
      na.idx <- which(is.na(dat.abundances2[i,insulin.idx]))
      dat.abundances2[i, insulin.idx[na.idx]] <- mean(dat.abundances2[i, insulin.idx], na.rm=T)
    }
}
nrow(dat.abundances)
nrow(na.omit(dat.abundances))   # na.omit() removes rows with at least on 'NA'
nrow(na.omit(dat.abundances2))

dat.abundances2 <- na.omit(dat.abundances2)   # delete phosphopeptides that still have missing values despite imputation

# **Identification of differential candidates**
Calculate p-values and fold changes in order to identify differential candidates (same as in part2).

In [ ]:
phospep.number <- nrow(dat.abundances2)

# define empty vectors for storing...
fc <- vector(length=phospep.number, mode="numeric")
p.val <- vector(length=phospep.number, mode="numeric")
p.val.adj <- vector(length=phospep.number, mode="numeric")

# adopt IDs of vector elements from row IDs of dat.abundances2
names(fc) <- rownames(dat.abundances2)
names(p.val) <- rownames(dat.abundances2)
names(p.val.adj) <- rownames(dat.abundances2)



# Calculate fold changes
for(i in 1:phospep.number){
    basal.mean <- mean(dat.abundances2[i, basal.idx])
    insulin.mean <- mean(dat.abundances2[i, insulin.idx])

    fc[i] <- insulin.mean / basal.mean
}



# Calculate p-values and adjusted p-values
for(i in 1:phospep.number){
    p.val[i] <- t.test(log2(dat.abundances2[i, basal.idx]), log2(dat.abundances2[i, insulin.idx]))$p.value
}
p.val.adj <- p.adjust(p.val, method="fdr")



# Give row numbers of candidates with both large log-fold change and low adj. p-value
diff.idx1 <- which(abs(log2(fc)) > 1)         # trick: via absolute value of log2(fc) we get both phosphopeptides with fc > 2 or 1/2
diff.idx2 <- which(p.val.adj < 0.05)          # gives row indices of phosphopeptides with adj. p-value < 0.05
diff.idx <- intersect(diff.idx1, diff.idx2)   # intersection gives row indices of differential candidates
print(length(diff.idx))

# Give row numbers of candidates with more stringent thresholds
diff.idx1 <- which(abs(log2(fc)) > 1.25)
diff.idx2 <- which(p.val.adj < 0.005)
diff.idx <- intersect(diff.idx1, diff.idx2)
print(length(diff.idx))

# **Prepare the list of interesting candidates**
For overrepresentation analysis a list of proteins or genes is needed. Thus, first we have to map the phosphopeptide IDs (row IDs) of our candidates to protein IDs (UniProt IDs). These are the phosphopeptide IDs of our candidates:

In [ ]:
rownames(dat.abundances2)[diff.idx]

Since the UniProt ID is already part of the phosphopeptide IDs we just need to extract them via an R function that employs regular expressions. This function is gsub(), which can substitute or delete parts of a given string specified by a regular expression.

In [ ]:
uniprot.ids <- c()
for(i in 1:length(diff.idx)){
  #print(paste0("Original: ", rownames(dat.abundances2)[diff.idx[i]]))
  tmp <- gsub("_peptide\\d+", "", rownames(dat.abundances2)[diff.idx[i]])
  #print(paste0("After removing '_peptide\\d+': ", tmp))

  tmp <- gsub("-\\d+", "", tmp)
  #print(paste0("After removing '-\\d+': ", tmp))

  #tmp <- print(strsplit(tmp, "; "))
  tmp <- strsplit(tmp, "; ")
  #print("After splitting:")
  #print(tmp[[1]])
  uniprot.ids <- c(uniprot.ids, tmp[[1]])

  #print("-----------------------")
}
uniprot.ids <- unique(uniprot.ids)
print(uniprot.ids)
print(length(uniprot.ids))

write.table(x=uniprot.ids, file="uniprot.ids.txt", quote=F, sep="\t", row.names=F, col.names=F)

In [ ]:
id.map <- read.table("https://raw.githubusercontent.com/ddz-icb/OmicsDataAnalysisCourse/main/data/idmapping_2023_11_21_to_GeneID.tsv",
                            header=T,
                            sep="\t")
head(id.map)

entrez.ids <- id.map[id.map$From %in% uniprot.ids,"To"]
print(entrez.ids)
length(entrez.ids)


# **STRING in R**
There is a relatively user-friendly R package in R for STRING network analysis. However, it is generally only of interest for automatic workflows for standard analyses and/or parameter optimizations because the STRING web application is usually more user-friendly.

In [ ]:
# Warning: Installation of the package takes ca. 6 minutes in Colab!
setRepositories(ind=1:5)
install.packages("STRINGdb")

library(STRINGdb)

Preparation of the UniProt IDs of our candidates as a data frame. Downloading the STRING interactions for our target species (mouse) with confidence threshold = 0.7. Finally, mapping our candidates to the downloaded interactions, discarding candidates without interactions.

In [ ]:
uniprot.ids.df <- data.frame("protein"=uniprot.ids)
string_db <- STRINGdb$new(species=10090, score_threshold=700)
candidates.mapped <- string_db$map(uniprot.ids.df, "protein", removeUnmappedRows=T)

Next, we want to keep only connected proteins and visualize them.

In [ ]:
# get STRING_ids of all proteins in the candidates.mapped object
hits <- candidates.mapped$STRING_id[1:length(diff.idx)]

# plot network without not connected proteins
options(repr.plot.width=20, repr.plot.height=20)
string_db$plot_network(hits)

Network visualization without not connected nodes:

In [ ]:
# get all interactions (above confidence threshold) that were found for them
interactions <- string_db$get_interactions(hits)
# get STRING_ids of proteins with interactions
connected <- unique(c(interactions$from, interactions$to))

# plot network without not connected proteins
options(repr.plot.width=20, repr.plot.height=20)
string_db$plot_network(connected)

As we know, we can carry out an ORA with STRING - this is also possible with the R package:

In [ ]:
ora.results <- string_db$get_enrichment(connected)
print(ora.results[ora.results$fdr<1.0e-15, c("category","term","description","fdr")])

To further/manually inspect the network it is also possible to get a link for the STRING web application and continue there.

In [ ]:
string_db$get_link(string_ids=connected, required_score=NULL, network_flavor="evidence")

# **WCGNA**
In order to perform weighted gene co-expression network analysis (WGCNA) in R the reference R package wgcna can been used.

In [ ]:
# Warning: Installation of the package takes ca. 6 minutes in Colab!
setRepositories(ind=1:5)
install.packages("WGCNA")

library(WGCNA)

Settings recommended by package creators:

In [ ]:
# The following setting is important, do not omit.
options(stringsAsFactors = FALSE);
# Allow multi-threading within WGCNA. This helps speed up certain calculations.
# At present this call is necessary for the code to work.
# Any error here may be ignored but you may want to update WGCNA if you see one.
# Caution: skip this line if you run RStudio or other third-party R environments.
# See note above.
enableWGCNAThreads()
# Load the data saved in the first part
#lnames = load(file = "FemaleLiver-01-dataInput.RData");
#The variable lnames contains the names of loaded variables.
#lnames

WCGNA provides "soft thresholding" as an alternative to "hard" correlation threshold for co-expression network construction. For this the "power" parameter needs to be optimized. This done in the following code:  

In [ ]:
# Choose a set of soft-thresholding powers
powers <- c(c(1:10), seq(from = 12, to=20, by=2))
# Call the network topology analysis function
sft <- pickSoftThreshold(t(dat.abundances2), powerVector = powers, verbose = 5)
#sft <- pickSoftThreshold(dat.abundances2[diff.idx,], powerVector = powers, verbose = 5)

Plot the soft threshold optimization results:

In [ ]:
# Plot the results:
#sizeGrWindow(9, 5)
par(mfrow = c(1,2));
options(repr.plot.width=20, repr.plot.height=10)
cex1 = 1.9;

# Scale-free topology fit index as a function of the soft-thresholding power
plot(sft$fitIndices[,1], -sign(sft$fitIndices[,3])*sft$fitIndices[,2],
xlab="Soft Threshold (power)",ylab="Scale Free Topology Model Fit,signed R^2",type="n",
main = paste("Scale independence"));
text(sft$fitIndices[,1], -sign(sft$fitIndices[,3])*sft$fitIndices[,2],
labels=powers,cex=cex1,col="red");
# this line corresponds to using an R^2 cut-off of h
abline(h=0.90,col="red")

# Mean connectivity as a function of the soft-thresholding power
plot(sft$fitIndices[,1], sft$fitIndices[,5],
xlab="Soft Threshold (power)",ylab="Mean Connectivity", type="n",
main = paste("Mean connectivity"))
text(sft$fitIndices[,1], sft$fitIndices[,5], labels=powers, cex=cex1,col="red")

Construct the co-expression network & identify modules using "power = 12" as this is the lowest power providing good results (above horizontal line in the above plot):

In [ ]:
# Runs 1 min in Google Colab
net <- blockwiseModules(t(dat.abundances2),
                        power = 12,
                        TOMType = "unsigned",
                        minModuleSize = 30,
                        reassignThreshold = 0,
                        mergeCutHeight = 0.25,
                        numericLabels = TRUE,
                        pamRespectsDendro = FALSE,
                        saveTOMs = TRUE,
                        saveTOMFileBase = "insulinMouseTOM",
                        verbose = 3)

How many modules were detected and how many phosphopeptides they have?

In [ ]:
table(net$colors)

Now let's visualize the modules as dendrograms:

In [ ]:
# Convert labels to colors for plotting
mergedColors <- labels2colors(net$colors)
# Plot the dendrogram and the module colors underneath
plotDendroAndColors(net$dendrograms[[1]],
                    mergedColors[net$blockGenes[[1]]],
                    "Module colors",
                    dendroLabels = FALSE,
                    hang = 0.03,
                    addGuide = TRUE,
                    guideHang = 0.05)

plotDendroAndColors(net$dendrograms[[2]],
                    mergedColors[net$blockGenes[[2]]],
                    "Module colors",
                    dendroLabels = FALSE,
                    hang = 0.03,
                    addGuide = TRUE,
                    guideHang = 0.05)

plotDendroAndColors(net$dendrograms[[3]],
                    mergedColors[net$blockGenes[[3]]],
                    "Module colors",
                    dendroLabels = FALSE,
                    hang = 0.03,
                    addGuide = TRUE,
                    guideHang = 0.05)

Finally, let's inspect a specific module - e.g. the "magenta" module. For this, give the peptide IDs of the module and map them to respective UniProt protein IDs. Then, write the UniProt IDs into a file in order to inspect the module e.g. in STRING to check, whether there is any known biological relevance.

In [ ]:
labels2colors(net$colors)

module.idx <- labels2colors(net$colors) == 'magenta'
print(net$colors[module.idx])
module.ids <- names(net$colors[module.idx])

uniprot.ids2 <- c()
for(i in 1:length(module.ids)){
  tmp <- gsub("_peptide\\d+", "", module.ids[i])
  tmp <- gsub("-\\d+", "", tmp)
  tmp <- strsplit(tmp, "; ")
  uniprot.ids2 <- c(uniprot.ids2, tmp[[1]])
}
uniprot.ids2 <- unique(uniprot.ids2)
print(uniprot.ids2)
print(length(uniprot.ids2))

write.table(x=uniprot.ids2, file="module.uniprot.ids.txt", quote=F, sep="\t", row.names=F, col.names=F)